In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### The column names initially are : 

In [ ]:
ds = pd.read_csv('/kaggle/input/up-crime-data-year-2019/MR data - Compiled Data Set.csv')
ds.loc[[155]]

In [ ]:
dataset = pd.read_csv('/kaggle/input/up-crime-data-year-2019/MR data - Compiled Data Set.csv')
# dataset.head(0)
# for col in dataset.columns:
#     print(col,end='\n************\n')

In [ ]:
# Changing the names of the columns to something readable
new_columns = dataset.columns.values
# print(type(new_columns))
new_columns[0]='City'
new_columns[3]='murder_cause'
new_columns[10]='IPC crime act'
new_columns[4]='Child Victims'
new_columns[5]='Male Victims'
new_columns[6]='Female Victims'
new_columns[7]='Kidnaped Child Victims'
# print("\nAfter change :{}".format(new_columns))
#Re-Set the column names on the og dataset
dataset.columns = new_columns
# Dropping the irrelevant columns
dataset=dataset.drop(['Title','Text'],axis=1)
# Setting NaN values to 0
dataset=dataset.fillna(0)

In [ ]:
dataset.head()

In [ ]:
for i in range(1,12):
    dataset.insert(7+i,"Act{}".format(i),0)
# dataset.head()

In [ ]:
# dataset.columns

In [ ]:
dataset['IPC crime act']=dataset['IPC crime act'].astype('string')
crime_act_values = (dataset['IPC crime act'].values)

# Seperating the crimes | i.e   8,9,10  =>  ['8','9','10']
for j in range(len(crime_act_values)):
    acts_commited = crime_act_values[j].split(',')
#     print(acts_commited)
    for x in acts_commited:
        if x=='0':
            continue
        dataset.loc[int(j),'Act{}'.format(x.strip())]=1
        
# dataset.head()

In [ ]:
# Murder cause unique values
dataset.murder_cause.unique()

In [ ]:
# dataset.drop(['IPC crime act'],axis=1)

In [ ]:
dataset.head()


In [ ]:
city_wise=(dataset['City'].value_counts().to_dict())
city_wise

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#making city-wise crime report 
vals = list(city_wise.values())
name = list(city_wise.keys())

plt.figure(figsize=(15,15))
plt.subplot(2,2,1)

plt.pie(vals,labels=name,autopct='%.2f%%')
plt.title("City wise crime data",fontsize=20)

# plt.show()

# plt.bar(range(len(city_wise)),vals,tick_label=name)
# plt.title("City wise crime data")
# plt.xlabel("City")
# plt.ylabel("Crimes commited")
# plt.show()


# 
# 
murder_reasons = dataset['murder_cause'].value_counts().to_dict()
murder_reasons.pop(0)

m_val = list(murder_reasons.values())
m_key = list(murder_reasons.keys())
plt.figure(figsize=(15,15))
plt.subplot(2,2,2)
plt.bar(range(len(murder_reasons)),m_val,tick_label = m_key)
plt.ylabel("Number of crimes commited",fontsize=20)
plt.title("# of crimes v/s Common Reasons",fontsize=20)
plt.xticks(rotation=90,fontsize=20)
# plt.show()

# 
# 
my_dict={}
for i in range(1,12):
    my_dict["Act {}".format(i)]=dataset["Act{}".format(i)].sum() 

act_val = list(my_dict.values())
act_key = list(my_dict.keys())
plt.subplot(2,2,3)
# plt.figure(figsize=(15,15))
plt.bar(range(len(my_dict)),act_val,tick_label = act_key)
plt.ylabel("Number of crimes commited",fontsize=20)
plt.title("IPC Act wise crimes commited ",fontsize=20)
plt.xticks(rotation=90,fontsize=15)

plt.show()

In [ ]:
# Number of people affected due to different IPC acts :

# dictionary (act -> (child , adult))
count = {}
for i in range(1,12):
    count[i]=[0,0]
for i in range(len(dataset['City'])):
    for actno in range(1,12):
        if dataset.loc[i,"Act{}".format(actno)]!=0:
            count[actno][0]+=int(dataset.loc[i,"Number of childs invoved"])
            count[actno][1]+=int(dataset.loc[i,"Number of adult victims"])


In [ ]:
#seperating the child & adult lists :
child_affected=[]
adults_affected=[]
for i in range(1,12):
    child_affected.append(count[i][0])
    adults_affected.append(count[i][1])
# print(child_affected)
# print(adults_affected)
index=np.arange(11)
width=0.3
plt.figure(figsize=(9,9))
plt.bar(index,child_affected,width,color='red',label = 'Children affected')
plt.bar(index+width,adults_affected,width,color='blue',label = 'Adults affected')
plt.xticks(index+width/2,act_key,rotation=90,fontsize=13)
plt.legend(loc='best',prop={'size':20})
plt.xlabel("Acts",fontsize=16)
plt.ylabel("# of people affected",fontsize=16)
plt.title("Number of people affected by different acts",fontsize=20)
plt.show()